In [42]:
import os
import sys
import json
import time
import warnings
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any
from dataclasses import dataclass

import pandas as pd
import numpy as np

# ML
try:
    from prophet import Prophet
    from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
    ML_OK = True
except:
    ML_OK = False
    print("⚠️  Install: pip install prophet scikit-learn")

# Visualization
try:
    import plotly.graph_objects as go
    VIZ_OK = True
except:
    VIZ_OK = False
    print("⚠️  Install: pip install plotly")

# Llama
LLAMA_OK = False
try:
    from langchain_community.llms import Ollama
    from langchain.callbacks.base import BaseCallbackHandler
    from langchain.schema import Document
    LLAMA_OK = True
except:
    print("⚠️  Llama packages missing (fallback mode)")

# RAG
RAG_OK = False
try:
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain_community.vectorstores import Chroma
    RAG_OK = True
except:
    print("⚠️  RAG packages missing")

warnings.filterwarnings('ignore')
np.random.seed(42)

print(f"\n{'='*70}")
print("SYSTEM CHECK")
print(f"{'='*70}")
print(f"ML:      {'✅' if ML_OK else '❌'}")
print(f"Viz:     {'✅' if VIZ_OK else '❌'}")
print(f"Llama:   {'✅' if LLAMA_OK else '❌'}")
print(f"RAG:     {'✅' if RAG_OK else '❌'}")
print(f"{'='*70}\n")

if not ML_OK:
    raise ImportError("CRITICAL: Install prophet and scikit-learn")



SYSTEM CHECK
ML:      ✅
Viz:     ✅
Llama:   ✅
RAG:     ✅



In [43]:
@dataclass
class Config:
    forecast_days: int = 90
    safety_stock: float = 1.1
    target_mape: float = 10.0
    lcs_forecast_w: float = 0.4
    lcs_payment_w: float = 0.3
    lcs_turnover_w: float = 0.3
    lcs_threshold: int = 80
    base_fee: float = 0.03
    fee_factor: float = 0.0005
    llama_model: str = "llama3.2"
    llama_url: str = "http://localhost:11434"
    llama_temp: float = 0.05
    llama_tokens: int = 800
    embed_model: str = "all-MiniLM-L6-v2"
    chunk_size: int = 400
    top_k: int = 3
    data_dir: str = "./sherpai_data"
    vector_dir: str = "./sherpai_vectors"
    output_dir: str = "./sherpai_output"
    
    def __post_init__(self):
        for d in [self.data_dir, self.vector_dir, self.output_dir]:
            Path(d).mkdir(parents=True, exist_ok=True)

CFG = Config()
print(f"✅ Config loaded\n")


✅ Config loaded



In [44]:
class StreamHandler(BaseCallbackHandler):
    def __init__(self):
        self.text = ""
        self.tokens = 0
        self.start = None
    
    def on_llm_start(self, *args, **kwargs):
        self.start = time.time()
        print("🧠 ", end="", flush=True)
    
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)
        self.text += token
        self.tokens += 1
    
    def on_llm_end(self, *args, **kwargs):
        elapsed = time.time() - self.start if self.start else 0
        print(f"\n\n[{self.tokens} tokens, {elapsed:.1f}s]")

print("✅ Stream handler ready\n")


✅ Stream handler ready



In [45]:
class LlamaEngine:
    def __init__(self, cfg: Config):
        self.cfg = cfg
        self.llm = None
        self.embeddings = None
        self.vectorstore = None
        self.ready = False
    
    def initialize(self) -> bool:
        if not LLAMA_OK:
            print("⚠️  Llama unavailable - using fallback\n")
            return False
        
        print(f"\n{'='*70}")
        print("INITIALIZING LLAMA")
        print(f"{'='*70}\n")
        
        try:
            print("[1/3] Connecting...")
            self.llm = Ollama(
                base_url=self.cfg.llama_url,
                model=self.cfg.llama_model,
                temperature=self.cfg.llama_temp,
                num_predict=self.cfg.llama_tokens
            )
            print(f"      ✅ Connected\n")
        except Exception as e:
            print(f"      ❌ Failed: {e}\n")
            return False
        
        try:
            print("[2/3] Testing...")
            resp = self.llm.invoke("Say: OK")
            if resp and len(resp.strip()) > 0:
                print(f"      ✅ Working\n")
            else:
                print(f"      ❌ Empty response\n")
                return False
        except Exception as e:
            print(f"      ❌ Test failed: {e}\n")
            return False
        
        if RAG_OK:
            try:
                print("[3/3] Loading embeddings...")
                self.embeddings = HuggingFaceEmbeddings(
                    model_name=self.cfg.embed_model,
                    model_kwargs={'device': 'cpu'}
                )
                print(f"      ✅ Ready\n")
            except Exception as e:
                print(f"      ⚠️  RAG disabled: {e}\n")
        
        self.ready = True
        print(f"{'='*70}")
        print("✅ LLAMA OPERATIONAL")
        print(f"{'='*70}\n")
        return True
    
    def create_vectorstore(self, docs: List[Document]) -> bool:
        if not self.embeddings:
            return False
        try:
            print(f"📚 Creating vectorstore...")
            self.vectorstore = Chroma.from_documents(
                documents=docs,
                embedding=self.embeddings,
                persist_directory=self.cfg.vector_dir
            )
            self.vectorstore.persist()
            print(f"   ✅ Done\n")
            return True
        except:
            return False
    
    def analyze(self, forecast: Dict, lcs: Dict, finance: Dict, 
                suppliers: List[Dict]) -> str:
        if not self.ready:
            return self._fallback(forecast, lcs, finance, suppliers)
        
        try:
            ctx = self._make_context(forecast, lcs, finance, suppliers)
            prompt = self._build_prompt(ctx)
            
            print("\n🧭 sherpAI analyzing...\n")
            handler = StreamHandler()
            self.llm.invoke(prompt, callbacks=[handler])
            
            return f"""
╔══════════════════════════════════════════════════════════════════════╗
║                   🧭 SHERPAI RECOMMENDATION                          ║
╚══════════════════════════════════════════════════════════════════════╝

{handler.text}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Powered by {self.cfg.llama_model}
"""
        except Exception as e:
            print(f"⚠️  AI failed: {e}")
            return self._fallback(forecast, lcs, finance, suppliers)
    
    def _make_context(self, f: Dict, l: Dict, fin: Dict, s: List[Dict]) -> str:
        best = min(s, key=lambda x: x['price'])
        return f"""FORECAST: {f['optimal_quantity']:,} units (expected {f['expected_demand']:,})
LCS: {l['lcs_score']}/100 ({l['risk_category']})
FINANCE: {fin['status']}
SUPPLIER: {best['name']} @ ${best['price']:.2f}/unit"""
    
    def _build_prompt(self, ctx: str) -> str:
        sys = "You are sherpAI. Analyze and provide: 1) Forecast confidence 2) Order recommendation 3) Financing advice 4) Risks 5) Action steps. Be concise."
        return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{sys}<|eot_id|><|start_header_id|>user<|end_header_id|>

{ctx}

Analyze.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    def _fallback(self, f: Dict, l: Dict, fin: Dict, s: List[Dict]) -> str:
        best = min(s, key=lambda x: x['price'])
        return f"""
╔══════════════════════════════════════════════════════════════════════╗
║              📊 SHERPAI RECOMMENDATION (Fallback)                    ║
╚══════════════════════════════════════════════════════════════════════╝

⚠️  AI unavailable

RECOMMENDATION:
• Order: {f['optimal_quantity']:,} units
• Supplier: {best['name']}
• Expected: {f['expected_demand']:,} units
• Risk: {f.get('stockout_risk', 'Unknown')}

FINANCING:
• Status: {fin['status']}
• LCS: {l['lcs_score']}/100
"""

llama = LlamaEngine(CFG)
LLAMA_READY = llama.initialize()



INITIALIZING LLAMA

[1/3] Connecting...
      ✅ Connected

[2/3] Testing...
      ✅ Working

[3/3] Loading embeddings...
      ⚠️  RAG disabled: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

✅ LLAMA OPERATIONAL



In [46]:
def generate_sales(start: str, end: str) -> pd.DataFrame:
    dates = pd.date_range(start, end, freq='D')
    n = len(dates)
    
    trend = np.linspace(100, 150, n)
    yearly = 30 * np.sin(2 * np.pi * np.arange(n) / 365.25)
    weekly = 15 * np.sin(2 * np.pi * np.arange(n) / 7)
    promos = np.zeros(n)
    promo_idx = np.random.choice(n, int(n * 0.08), replace=False)
    promos[promo_idx] = np.random.uniform(30, 70, len(promo_idx))
    noise = np.random.normal(0, 10, n)
    
    sales = np.maximum(trend + yearly + weekly + promos + noise, 5)
    
    return pd.DataFrame({
        'date': dates,
        'sales': sales,
        'revenue': sales * np.random.uniform(20, 30, n)
    }).set_index('date')

sales_data = generate_sales('2022-01-01', '2024-12-31')
print(f"✅ Sales data: {len(sales_data)} days\n")


✅ Sales data: 1096 days



In [47]:
class ForecastEngine:
    def __init__(self, cfg: Config):
        self.cfg = cfg
        self.model = None
        self.forecast = None
        self.metrics = {}
    
    def train(self, df: pd.DataFrame) -> Dict:
        print("\n🔧 Training forecast...")
        
        data = df.reset_index().rename(columns={'date': 'ds', 'sales': 'y'})
        data['ds'] = pd.to_datetime(data['ds'])
        
        self.model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.05,
            interval_width=0.95
        )
        self.model.fit(data)
        
        pred = self.model.predict(data)
        mape = mean_absolute_percentage_error(data['y'], pred['yhat']) * 100
        rmse = np.sqrt(mean_squared_error(data['y'], pred['yhat']))
        
        self.metrics = {'mape': mape, 'rmse': rmse, 'samples': len(data)}
        
        print(f"   {'✅ Excellent' if mape < self.cfg.target_mape else '✅ Good'}")
        print(f"   MAPE: {mape:.2f}%, RMSE: {rmse:.2f}\n")
        
        return self.metrics
    
    def predict(self, days: int = None) -> pd.DataFrame:
        days = days or self.cfg.forecast_days
        future = self.model.make_future_dataframe(periods=days)
        self.forecast = self.model.predict(future)
        print(f"✅ Forecast: {days} days\n")
        return self.forecast
    
    def calc_order(self, stock: int = 0) -> Dict:
        future = self.forecast[self.forecast['ds'] > datetime.now()].head(44)
        exp = int(future['yhat'].sum())
        opt = int(exp * self.cfg.safety_stock)
        lo = max(0, int(future['yhat_lower'].sum()))
        hi = int(future['yhat_upper'].sum())
        
        var = (hi - lo) / exp if exp > 0 else 0
        risk = "🟢 LOW" if var < 0.2 else "🟡 MED" if var < 0.4 else "🔴 HIGH"
        
        return {
            'optimal_quantity': opt,
            'expected_demand': exp,
            'current_stock': stock,
            'lower_bound': lo,
            'upper_bound': hi,
            'forecast_period_days': len(future),
            'stockout_risk': risk,
            'safety_stock_units': opt - exp
        }
    
    def visualize(self, df: pd.DataFrame) -> Optional[go.Figure]:
        if not VIZ_OK:
            print("⚠️  Plotly not available")
            return None
        
        fig = go.Figure()
        
        hist = df.reset_index()
        fig.add_trace(go.Scatter(
            x=hist['date'], y=hist['sales'],
            mode='markers', name='Actual',
            marker=dict(size=3, color='blue', opacity=0.5)
        ))
        
        fut = self.forecast[self.forecast['ds'] > datetime.now()]
        fig.add_trace(go.Scatter(
            x=fut['ds'], y=fut['yhat'],
            mode='lines', name='Forecast',
            line=dict(color='red', width=2)
        ))
        
        fig.add_trace(go.Scatter(
            x=fut['ds'], y=fut['yhat_upper'],
            mode='lines', line=dict(width=0),
            showlegend=False
        ))
        
        fig.add_trace(go.Scatter(
            x=fut['ds'], y=fut['yhat_lower'],
            mode='lines', line=dict(width=0),
            fill='tonexty', fillcolor='rgba(255,0,0,0.2)',
            name='95% CI'
        ))
        
        # 🐛 FIX APPLIED: Separate line and annotation logic to avoid Plotly's internal TypeError.
        today_iso = datetime.now().strftime('%Y-%m-%d')
        
        # 1. Add the vertical line using add_shape
        fig.add_shape(
            type="line",
            x0=today_iso, x1=today_iso, 
            y0=0, y1=1,
            yref="paper", # Use paper coordinates for Y to span the entire plot area
            line=dict(dash="dash", color="gray", width=1)
        )

        # 2. Add the annotation (text) separately using add_annotation
        fig.add_annotation(
            text="Today",
            x=today_iso, 
            y=1.02, # Position text slightly above the top edge (y=1.0)
            yref="paper",
            showarrow=False,
            font=dict(color="gray"),
            xanchor="center"
        )
        # -------------------------------------------------------------------------

        fig.update_layout(
            title="📈 sherpAI Forecast",
            xaxis_title="Date",
            yaxis_title="Sales",
            height=500,
            template='plotly_white'
        )
        
        return fig

print("✅ Forecast engine ready\n")


✅ Forecast engine ready



In [35]:
class LCSEngine:
    def __init__(self, cfg: Config):
        self.cfg = cfg
    
    def calculate(self, mape: float, payment: List[bool] = None, 
                 turnover: float = None) -> Dict:
        fs = max(0, 100 - (mape * 2))
        if mape < 5:
            fs = min(100, fs + (5 - mape) * 2)
        
        ps = (sum(payment) / len(payment) * 100) if payment else 60.0
        
        if turnover:
            if 6 <= turnover <= 12:
                ts = 100.0
            elif turnover < 6:
                ts = max(0, 100 - (6 - turnover) * 15)
            else:
                ts = max(0, 100 - (turnover - 12) * 8)
        else:
            ts = 50.0
        
        lcs = int(
            fs * self.cfg.lcs_forecast_w +
            ps * self.cfg.lcs_payment_w +
            ts * self.cfg.lcs_turnover_w
        )
        
        if lcs >= 90:
            risk = "Excellent (Very Low Risk)"
        elif lcs >= 80:
            risk = "Good (Low Risk)"
        elif lcs >= 70:
            risk = "Fair (Medium Risk)"
        elif lcs >= 60:
            risk = "Marginal (Med-High Risk)"
        else:
            risk = "Poor (High Risk)"
        
        return {
            'lcs_score': lcs,
            'forecast_accuracy_score': int(fs),
            'historical_performance_score': int(ps),
            'inventory_turnover_score': int(ts),
            'risk_category': risk,
            'meets_threshold': lcs >= self.cfg.lcs_threshold
        }

class FinanceEngine:
    def __init__(self, cfg: Config):
        self.cfg = cfg
    
    def calculate(self, po_val: float, lcs: int) -> Dict:
        if lcs < self.cfg.lcs_threshold:
            return {
                'status': 'REJECTED',
                'reason': f'LCS {lcs} < {self.cfg.lcs_threshold}',
                'principal': 0
            }
        
        term = 60 if lcs >= 90 else 30
        rate = self.cfg.base_fee + (100 - lcs) * self.cfg.fee_factor
        fee = po_val * rate
        total = po_val + fee
        
        return {
            'status': 'APPROVED',
            'principal': po_val,
            'fee_rate': f'{rate * 100:.2f}%',
            'fee_amount': fee,
            'total_repayment': total,
            'term_days': term,
            'apr_equivalent': f'{(rate * 365 / term) * 100:.1f}%',
            'lcs_score': lcs
        }

lcs_engine = LCSEngine(CFG)
finance_engine = FinanceEngine(CFG)
print("✅ LCS & Finance ready\n")


✅ LCS & Finance engines ready



In [49]:
def get_suppliers() -> List[Dict]:
    return [
        {'id': 'SUP_001', 'name': 'Primary Supplier Co.', 
         'price': 25.00, 'moq': 100, 'lead_days': 14, 'reliability': 95},
        {'id': 'SUP_002', 'name': 'Backup Supplier Ltd.', 
         'price': 27.00, 'moq': 50, 'lead_days': 10, 'reliability': 88},
        {'id': 'SUP_003', 'name': 'Economy Supplier Inc.', 
         'price': 23.50, 'moq': 200, 'lead_days': 21, 'reliability': 82}
    ]

def select_supplier(suppliers: List[Dict], qty: int) -> Dict:
    valid = [s for s in suppliers if qty >= s['moq']]
    if not valid:
        return min(suppliers, key=lambda x: x['moq'])
    return min(valid, key=lambda s: (s['price'] * qty) / (s['reliability'] / 100))

print("✅ Suppliers ready\n")


✅ Suppliers ready



In [50]:
def prepare_docs(forecast_df: pd.DataFrame, metrics: Dict) -> List[Document]:
    """Convert forecast to documents"""
    
    docs = []
    future = forecast_df[forecast_df['ds'] > datetime.now()]
    
    # Summary
    summary = f"""Forecast: {future['yhat'].sum():.0f} units over {len(future)} days
Avg daily: {future['yhat'].mean():.1f} units
MAPE: {metrics['mape']:.2f}%
Training: {metrics['samples']} samples"""
    
    docs.append(Document(page_content=summary))
    
    # Weekly
    future_copy = future.copy()
    future_copy['week'] = pd.to_datetime(future_copy['ds']).dt.isocalendar().week
    weekly = future_copy.groupby('week').agg({
        'yhat': 'sum',
        'yhat_lower': 'sum',
        'yhat_upper': 'sum'
    }).reset_index()
    
    for _, row in weekly.head(6).iterrows():
        week_text = f"Week {int(row['week'])}: {row['yhat']:.0f} units expected"
        docs.append(Document(page_content=week_text))
    
    # Risk
    var = (future['yhat_upper'] - future['yhat_lower']).mean() / future['yhat'].mean()
    risk_text = f"Variability: {var:.1%}"
    docs.append(Document(page_content=risk_text))
    
    return docs

print("✅ RAG docs ready\n")


✅ RAG docs ready



In [51]:
class SherpAI:
    def __init__(self, cfg: Config, llama_engine: LlamaEngine):
        self.cfg = cfg
        self.llama = llama_engine
        self.forecast = ForecastEngine(cfg)
        self.lcs = lcs_engine
        self.finance = finance_engine
    
    def analyze(self, sales_df: pd.DataFrame, stock: int = 50) -> Dict:
        print(f"\n{'='*70}")
        print("🧭 SHERPAI ANALYSIS")
        print(f"{'='*70}\n")
        
        # Train
        print("[1/7] Training...")
        metrics = self.forecast.train(sales_df)
        
        # Forecast
        print("[2/7] Forecasting...")
        forecast_df = self.forecast.predict()
        
        # Order
        print("[3/7] Calculating order...")
        order = self.forecast.calc_order(stock)
        print(f"      Optimal: {order['optimal_quantity']:,} units\n")
        
        # Suppliers
        print("[4/7] Evaluating suppliers...")
        suppliers = get_suppliers()
        best = select_supplier(suppliers, order['optimal_quantity'])
        print(f"      Best: {best['name']}\n")
        
        # LCS
        print("[5/7] Calculating LCS...")
        lcs_data = self.lcs.calculate(metrics['mape'], [True]*10, 8.5)
        print(f"      LCS: {lcs_data['lcs_score']}/100\n")
        
        # Finance
        print("[6/7] Calculating financing...")
        po_val = order['optimal_quantity'] * best['price']
        financing = self.finance.calculate(po_val, lcs_data['lcs_score'])
        print(f"      Status: {financing['status']}\n")
        
        # RAG
        if self.llama.ready and self.llama.embeddings:
            print("[7/7] Indexing for RAG...")
            docs = prepare_docs(forecast_df, metrics)
            self.llama.create_vectorstore(docs)
        
        # AI recommendation
        print("[7/7] Generating recommendation...")
        recommendation = self.llama.analyze(order, lcs_data, financing, suppliers)
        
        print(f"\n{'='*70}")
        print("✅ COMPLETE")
        print(f"{'='*70}\n")
        
        return {
            'sales_data': sales_df,
            'forecast_metrics': metrics,
            'forecast': forecast_df,
            'order_data': order,
            'suppliers': suppliers,
            'best_supplier': best,
            'lcs_data': lcs_data,
            'financing': financing,
            'recommendation': recommendation,
            'po_value': po_val
        }
    
    def visualize(self, results: Dict):
        return self.forecast.visualize(results['sales_data'])

sherp = SherpAI(CFG, llama)
print("✅ SherpAI system ready\n")


✅ SherpAI system ready



In [52]:
def run_sherpai():
    """Execute complete analysis"""
    
    print("""
╔══════════════════════════════════════════════════════════════════════╗
║                        🧭 sherpAI v1.0                               ║
╚══════════════════════════════════════════════════════════════════════╝
    """)
    
    results = sherp.analyze(sales_data, stock=75)
    
    print(results['recommendation'])
    
    print("\n📊 Generating visualization...")
    chart = sherp.visualize(results)
    
    print(f"\n{'='*70}")
    print("📈 KEY METRICS")
    print(f"{'='*70}")
    print(f"Forecast MAPE:        {results['forecast_metrics']['mape']:.2f}%")
    print(f"LCS Score:            {results['lcs_data']['lcs_score']}/100")
    print(f"Risk Category:        {results['lcs_data']['risk_category']}")
    print(f"Optimal Order:        {results['order_data']['optimal_quantity']:,} units")
    print(f"Expected Demand:      {results['order_data']['expected_demand']:,} units")
    print(f"PO Value:             ${results['po_value']:,.2f}")
    print(f"Financing Status:     {results['financing']['status']}")
    print(f"Best Supplier:        {results['best_supplier']['name']}")
    print(f"Stockout Risk:        {results['order_data']['stockout_risk']}")
    print(f"{'='*70}")
    
    print("\n💡 NEXT STEPS:")
    print(f"{'='*70}")
    if chart:
        print("1. View chart:        chart.show()")
    print("2. Export results:    export_results(results)")
    print("3. Access data:       results['forecast'].tail(30)")
    print(f"{'='*70}\n")
    
    return results, chart

def export_results(results: Dict, dir: str = None):
    """Export results to files"""
    
    output_dir = dir or CFG.output_dir
    Path(output_dir).mkdir(exist_ok=True)
    
    results['forecast'].to_csv(f"{output_dir}/forecast.csv", index=False)
    
    with open(f"{output_dir}/recommendation.txt", 'w') as f:
        f.write(results['recommendation'])
    
    summary = {
        'timestamp': datetime.now().isoformat(),
        'forecast_mape': results['forecast_metrics']['mape'],
        'lcs_score': results['lcs_data']['lcs_score'],
        'risk_category': results['lcs_data']['risk_category'],
        'optimal_quantity': results['order_data']['optimal_quantity'],
        'expected_demand': results['order_data']['expected_demand'],
        'po_value': results['po_value'],
        'financing_status': results['financing']['status'],
        'best_supplier': results['best_supplier']['name'],
        'stockout_risk': results['order_data']['stockout_risk']
    }
    
    with open(f"{output_dir}/summary.json", 'w') as f:
        json.dump(summary, f, indent=2)
    
    print(f"\n✅ Results exported to: {output_dir}/")
    print("   • forecast.csv")
    print("   • recommendation.txt")
    print("   • summary.json\n")

print("✅ Main execution ready\n")

# ═══════════════════════════════════════════════════════════════════════
# FINAL STATUS
# ═══════════════════════════════════════════════════════════════════════

print(f"{'='*70}")
print("✅ SHERPAI FULLY LOADED")
print(f"{'='*70}")
print("\nSTATUS:")
print(f"  ML/Forecasting:    {'✅' if ML_OK else '❌'}")
print(f"  Visualization:     {'✅' if VIZ_OK else '❌'}")
print(f"  Llama Engine:      {'✅' if LLAMA_READY else '⚠️  Fallback'}")
print(f"  RAG System:        {'✅' if RAG_OK and llama.embeddings else '⚠️  Disabled'}")
print(f"  Data Generated:    ✅ {len(sales_data)} days")

print(f"\n{'='*70}")

if LLAMA_READY:
    print("🎉 SHERPAI READY")
    print(f"{'='*70}")
    print("\n🚀 TO RUN:")
    print(">>> results, chart = run_sherpai()")
    if VIZ_OK:
        print(">>> chart.show()")
else:
    print("⚠️  Running in fallback mode")
    print(f"{'='*70}")
    print("\n🚀 TO RUN:")
    print(">>> results, chart = run_sherpai()")

print(f"{'='*70}")
print("\n✅ Ready! Execute run_sherpai() to start.\n") 

✅ Main execution ready

✅ SHERPAI FULLY LOADED

STATUS:
  ML/Forecasting:    ✅
  Visualization:     ✅
  Llama Engine:      ✅
  RAG System:        ⚠️  Disabled
  Data Generated:    ✅ 1096 days

🎉 SHERPAI READY

🚀 TO RUN:
>>> results, chart = run_sherpai()
>>> chart.show()

✅ Ready! Execute run_sherpai() to start.

